In [1]:
### Import packages for preprocessing

import pandas as pd
import string
import spacy
import nltk
import re
nltk.download('stopwords')

## Import packages to create absolute file path & make code independent of operating system

from pathlib import Path
import os.path
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/HenriekeMax/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
### Verify that spaCy was installed and that models and data were successfully downloaded

nlp = spacy.load('en_core_web_sm')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [3]:
### CSV-data into a dataframe

base_path = Path("__file__").parent

full_path = (base_path / "../../data/raw/stackoverflow_raw.csv").resolve()

df_stackoverflow = pd.read_csv(os.path.join(full_path))

NameError: name 'Path' is not defined

In [4]:
df_stackoverflow.head()

NameError: name 'df_stackoverflow' is not defined

In [5]:
df_stackoverflow.shape

NameError: name 'df_stackoverflow' is not defined

In [6]:
df_stackoverflow.info()

NameError: name 'df_stackoverflow' is not defined

In [7]:
df_stackoverflow.score.value_counts()

NameError: name 'df_stackoverflow' is not defined

In [8]:
# Sample from main dataset to test pre-processing code on less data

df_stackoverflow_sample = df_stackoverflow.sample(n = 10000, random_state = 1)

NameError: name 'df_stackoverflow' is not defined

In [9]:
df_stackoverflow_sample.shape

NameError: name 'df_stackoverflow_sample' is not defined

In [10]:
### Save sample of dataframe to a csv file

base_path = Path("__file__").parent
full_path = (base_path / "../../data/raw/stackoverflow_raw_sample.csv").resolve()

NameError: name 'Path' is not defined

In [11]:
df_stackoverflow_sample.to_csv(os.path.join(full_path))

NameError: name 'df_stackoverflow_sample' is not defined

In [12]:
df_stackoverflow_sample = pd.read_csv(os.path.join(full_path))

NameError: name 'full_path' is not defined

In [13]:
stopwords = nltk.corpus.stopwords.words('english')

NameError: name 'nltk' is not defined

In [14]:
print(stopwords)

NameError: name 'stopwords' is not defined

In [15]:
# Get rid of extras, split, and join for text,title, and cleaned paragraphs
def preprocess(x):
    x = re.sub(r'<code>.*?</code>', '', x.lower())
    x = re.sub(r'<p>', '', x.lower())
    x = re.sub('[^a-z\s]', '', x.lower())
    x = [w for w in x.split() if w not in stopwords]
    return ' '.join(x)

# add lemmatization?

In [16]:
# df_stackoverflow['qtitle']
df_stackoverflow_sample['answer_text_nocode'] = df_stackoverflow_sample['answer_text'].apply(preprocess)

NameError: name 'df_stackoverflow_sample' is not defined

In [17]:
df_stackoverflow_sample['answer_text_nocode'].head()

NameError: name 'df_stackoverflow_sample' is not defined

In [19]:
### Tokenization


In [21]:
### Lemmatization



In [22]:
### Part of speech tagging



In [23]:
### Dependency parsing



In [24]:
### Word vectorization

